# What assessments are the Instructors using?
* When are things due?
* How long are things open?
* How many points are they worth?
* How many students does this affect?


The query:
```sql
SELECT COURSE_ID,
       REPLACE(COURSE_NAME,',',' ') COURSE_NAME,
       emails,
       userNames,
       start_date,
       end_date,
       ROUND((end_date - start_date) * 24, 2) hours_open,
       DUE_DATE,
       TITLE,
       AVAILABLE_IND,
       POSSIBLE,
       students,
       CNTHNDLR_HANDLE,
       RULE_TYPE,
       PK1,
       DTCREATED,
       DTMODIFIED,
       MULTIPLE_ATTEMPTS,
       force_completion_IND,
       time_limit,
       questions
FROM (SELECT cm.COURSE_ID,
       cm.COURSE_NAME,
       CASE WHEN ac.START_DATE IS NOT NULL THEN ac.START_DATE ELSE cc.START_DATE END start_date,
       CASE WHEN ac.END_DATE IS NOT NULL THEN ac.END_DATE ELSE cc.END_DATE END end_date,
       gm.DUE_DATE,
       cc.PK1,
       cc.TITLE,
       cc.AVAILABLE_IND,
       gm.POSSIBLE,
       students.students,
       cc.CNTHNDLR_HANDLE,
       ar.TITLE as ar_title,
       ar.RULE_TYPE,
       ar.PK1 as ar_pk1,
       cc.DTCREATED,
       cc.DTMODIFIED,
       gm.MULTIPLE_ATTEMPTS,
       ca.FORCE_COMPLETION_IND,
       ca.time_limit,
       questions.questions,
       instructors.emails emails,
       instructors.userNames userNames,
       rank() over (partition by ar.PK1 order by ar.TITLE desc) rnk
    FROM BB_BB60.COURSE_CONTENTS cc
    JOIN BB_BB60.COURSE_MAIN cm on cc.CRSMAIN_PK1 = cm.PK1
    JOIN BB_BB60.GRADEBOOK_MAIN gm on cc.PK1 = gm.COURSE_CONTENTS_PK1
    LEFT JOIN BB_BB60.QTI_ASI_DATA qti on gm.QTI_ASI_DATA_PK1 = qti.PK1
    LEFT JOIN BB_BB60.COURSE_ASSESSMENT ca on qti.PK1 = ca.QTI_ASI_DATA_PK1
    LEFT JOIN BB_BB60.AVL_RULE ar on cc.PK1 = ar.COURSE_CONTENTS_PK1
    LEFT JOIN BB_BB60.AVL_CRIT ac on ar.PK1 = ac.AVL_RULE_PK1
    left join (select crsmain_pk1, count(pk1) students
    from bb_bb60.COURSE_USERS
    where role='S'
    and row_status = 0
    group by crsmain_pk1) students on students.crsmain_pk1 = cm.pk1
    left JOIN (select cu.CRSMAIN_PK1,
                      listagg(u.email, ';') WITHIN GROUP (order by u.email) emails,
                      listagg(u.user_id, ';') WITHIN GROUP (order by u.email) userNames
        from bb_bb60.COURSE_USERS cu join bb_bb60.USERS u on cu.USERS_PK1 = u.PK1
        WHERE (role='P' OR role='T') and
              cu.ROW_STATUS = 0
              group by cu.CRSMAIN_PK1) instructors on instructors.crsmain_pk1 = cm.pk1
    LEFT JOIN (SELECT qti1.PK1 pk1, COUNT(qti3.pk1) questions
        from bb_bb60.QTI_ASI_DATA qti1
        JOIN bb_bb60.QTI_ASI_DATA qti2 on qti1.PK1 = qti2.PARENT_PK1
        JOIN bb_bb60.QTI_ASI_DATA qti3 on qti2.PK1 = qti3.PARENT_PK1
    WHERE qti1.PARENT_PK1 IS NULL AND QTI3.BBMD_ASI_TYPE = 3
    GROUP BY qti1.PK1) questions on gm.QTI_ASI_DATA_PK1 = questions.pk1
     WHERE cm.COURSE_ID LIKE '4202%'
      and cc.CNTHNDLR_HANDLE IN ('resource/x-bb-asmt-survey-link',
                                 'resource/x-bb-assignment',
                                 'resource/x-bb-asmt-test-link')
  and (ar.Title is NULL or ar.Title != 'Test Option Exception Rule')
  and (gm.DUE_DATE > sysdate or gm.DUE_DATE IS NULL))
    WHERE rnk = 1
ORDER BY DUE_DATE;
```

In [1]:
from sqlalchemy import create_engine
from os import getenv as ge
from dotenv import load_dotenv
import pandas as pd

load_dotenv()
bbuser, bbpass, bbhost = ge('bbuser'), ge('bbpass'), ge('bbhost')
con_string = f'oracle+cx_oracle://{bbuser}:{bbpass}@{bbhost}'
engine = create_engine(con_string, max_identifier_length=128)

In [10]:
query = """
SELECT cm.COURSE_ID,
       cm.COURSE_NAME,
       CASE WHEN ac.START_DATE IS NOT NULL THEN ac.START_DATE ELSE cc.START_DATE END start_date,
       CASE WHEN ac.END_DATE IS NOT NULL THEN ac.END_DATE ELSE cc.END_DATE END end_date,
       gm.DUE_DATE,
       cc.PK1,
       cc.TITLE,
       cc.AVAILABLE_IND,
       cc.CNTHNDLR_HANDLE,
       cc.DTCREATED,
       cc.DTMODIFIED
    FROM BB_BB60.COURSE_CONTENTS cc
    JOIN BB_BB60.COURSE_MAIN cm on cc.CRSMAIN_PK1 = cm.PK1
    JOIN BB_BB60.GRADEBOOK_MAIN gm on cc.PK1 = gm.COURSE_CONTENTS_PK1
    
    LEFT JOIN BB_BB60.AVL_RULE ar on cc.PK1 = ar.COURSE_CONTENTS_PK1
    LEFT JOIN BB_BB60.AVL_CRIT ac on ar.PK1 = ac.AVL_RULE_PK1
    
    WHERE cm.COURSE_ID LIKE '4206-%'
      and cc.CNTHNDLR_HANDLE IN ('resource/x-bb-asmt-survey-link',
                                 'resource/x-bb-assignment',
                                 'resource/x-bb-asmt-test-link')
      and (gm.DUE_DATE > sysdate or gm.DUE_DATE IS NULL)
    ORDER BY gm.DUE_DATE
"""
pd.read_sql(query, con=engine).head()

,course_id,course_name,start_date,end_date,due_date,pk1,title,available_ind,cnthndlr_handle,dtcreated,dtmodified
0,4206-74142,2020Summer-MATH 220 Applied Differential Equat...,NaT,NaT,2020-07-13 16:00:00,8295222,Quiz 13_Due_7_13,Y,resource/x-bb-asmt-test-link,2020-05-17 09:50:21,2020-07-11 13:33:38
1,4206-74142,2020Summer-MATH 220 Applied Differential Equat...,2020-07-13 05:00:00,2020-07-13 09:00:00,2020-07-13 16:00:00,8295222,Quiz 13_Due_7_13,Y,resource/x-bb-asmt-test-link,2020-05-17 09:50:21,2020-07-11 13:33:38
2,4206-74142,2020Summer-MATH 220 Applied Differential Equat...,NaT,NaT,2020-07-13 16:00:00,8295222,Quiz 13_Due_7_13,Y,resource/x-bb-asmt-test-link,2020-05-17 09:50:21,2020-07-11 13:33:38
3,4206-74142,2020Summer-MATH 220 Applied Differential Equat...,2020-07-13 16:00:00,2020-07-13 18:00:00,2020-07-13 16:00:00,8295222,Quiz 13_Due_7_13,Y,resource/x-bb-asmt-test-link,2020-05-17 09:50:21,2020-07-11 13:33:38
4,4206-74142,2020Summer-MATH 220 Applied Differential Equat...,NaT,NaT,2020-07-13 16:00:00,8295222,Quiz 13_Due_7_13,Y,resource/x-bb-asmt-test-link,2020-05-17 09:50:21,2020-07-11 13:33:38


### Problems
There's multiple assignments? Same PK - but some have start and some don't? WHAT IS HAPPENING?
Availability rules are happening.  This is useful for an instructor trying to give more time to a student or group... but not to a developer trying to pull out the 'default' availability.
* Why doesn't this work in csv?
* Which of these assignments are real?
* How long are these open?

In [16]:
query = """
SELECT COURSE_ID,
       REPLACE(COURSE_NAME,',',' ') COURSE_NAME, --removed commas from course names
       start_date,
       end_date,
       ROUND((end_date - start_date) * 24, 2) hours_open, -- added calculation
       DUE_DATE,
       TITLE,
       AVAILABLE_IND,
       CNTHNDLR_HANDLE,
       PK1,
       DTCREATED,
       DTMODIFIED,
       rnk  -- to use this in the where clause, it's easiest to pull it directly out of a subquery
FROM(SELECT cm.COURSE_ID,
       cm.COURSE_NAME,
       CASE WHEN ac.START_DATE IS NOT NULL THEN ac.START_DATE ELSE cc.START_DATE END start_date,
       CASE WHEN ac.END_DATE IS NOT NULL THEN ac.END_DATE ELSE cc.END_DATE END end_date,
       gm.DUE_DATE,
       cc.PK1,
       cc.TITLE,
       cc.AVAILABLE_IND,
       cc.CNTHNDLR_HANDLE,
       cc.DTCREATED,
       cc.DTMODIFIED,
       rank() over (partition by ar.PK1 order by ar.TITLE desc) rnk
    FROM BB_BB60.COURSE_CONTENTS cc
    JOIN BB_BB60.COURSE_MAIN cm on cc.CRSMAIN_PK1 = cm.PK1
    JOIN BB_BB60.GRADEBOOK_MAIN gm on cc.PK1 = gm.COURSE_CONTENTS_PK1
    
    LEFT JOIN BB_BB60.AVL_RULE ar on cc.PK1 = ar.COURSE_CONTENTS_PK1
    LEFT JOIN BB_BB60.AVL_CRIT ac on ar.PK1 = ac.AVL_RULE_PK1
    
    WHERE cm.COURSE_ID LIKE '4206-%'
      and cc.CNTHNDLR_HANDLE IN ('resource/x-bb-asmt-survey-link',
                                 'resource/x-bb-assignment',
                                 'resource/x-bb-asmt-test-link')
      and (gm.DUE_DATE > sysdate or gm.DUE_DATE IS NULL)
      and (ar.Title is NULL or ar.Title != 'Test Option Exception Rule'))
WHERE rnk = 1
ORDER BY DUE_DATE
"""
pd.read_sql(query, con=engine).head()

,course_id,course_name,start_date,end_date,hours_open,due_date,title,available_ind,cnthndlr_handle,pk1,dtcreated,dtmodified,rnk
0,4206-74142,2020Summer-MATH 220 Applied Differential Equat...,2020-07-13 12:00:00,2020-07-13 18:00:00,6.0,2020-07-13 16:00:00,Quiz 13_Due_7_13,Y,resource/x-bb-asmt-test-link,8295222,2020-05-17 09:50:21,2020-07-11 13:33:38,1
1,4206-74142,2020Summer-MATH 220 Applied Differential Equat...,NaT,NaT,NaN,2020-07-13 16:00:00,Quiz 13_Due_7_13,Y,resource/x-bb-asmt-test-link,8295222,2020-05-17 09:50:21,2020-07-11 13:33:38,1
2,4206-75320,2020Summer-EAP 101 English for Academic Purpos...,2020-07-13 08:00:00,2020-07-13 16:30:00,8.5,2020-07-13 16:30:00,EAP101_MakingConnections3_Ch4_Reading Test Que...,Y,resource/x-bb-asmt-test-link,8411720,2020-07-10 11:10:38,2020-07-10 11:11:50,1
3,4206-75320,2020Summer-EAP 101 English for Academic Purpos...,2020-07-13 08:00:00,2020-07-13 16:30:00,8.5,2020-07-13 16:30:00,EAP101_MakingConnections3_Ch4_Reading Test Pas...,Y,resource/x-bb-asmt-test-link,8411713,2020-07-10 11:08:03,2020-07-10 11:10:17,1
4,4206-75313,2020Summer-EVRN 170 Introduction to Kansas Lan...,NaT,NaT,NaN,2020-07-13 17:00:00,Home Essay Revision,Y,resource/x-bb-assignment,8402290,2020-07-06 11:55:16,2020-07-06 11:55:45,1


In [17]:
q = """
SELECT COURSE_ID,
       REPLACE(COURSE_NAME,',',' ') COURSE_NAME,
       emails,
       userNames,
       start_date,
       end_date,
       ROUND((end_date - start_date) * 24, 2) hours_open,
       DUE_DATE,
       TITLE,
       AVAILABLE_IND,
       POSSIBLE,
       students,
       CNTHNDLR_HANDLE,
       RULE_TYPE,
       PK1,
       DTCREATED,
       DTMODIFIED,
       MULTIPLE_ATTEMPTS,
       force_completion_IND,
       time_limit,
       questions
FROM (SELECT cm.COURSE_ID,
       cm.COURSE_NAME,
       CASE WHEN ac.START_DATE IS NOT NULL THEN ac.START_DATE ELSE cc.START_DATE END start_date,
       CASE WHEN ac.END_DATE IS NOT NULL THEN ac.END_DATE ELSE cc.END_DATE END end_date,
       gm.DUE_DATE,
       cc.PK1,
       cc.TITLE,
       cc.AVAILABLE_IND,
       gm.POSSIBLE,
       students.students,
       cc.CNTHNDLR_HANDLE,
       ar.TITLE as ar_title,
       ar.RULE_TYPE,
       ar.PK1 as ar_pk1,
       cc.DTCREATED,
       cc.DTMODIFIED,
       gm.MULTIPLE_ATTEMPTS,
       ca.FORCE_COMPLETION_IND,
       ca.time_limit,
       questions.questions,
       instructors.emails emails,
       instructors.userNames userNames,
       rank() over (partition by ar.PK1 order by ar.TITLE desc) rnk
    FROM BB_BB60.COURSE_CONTENTS cc
    JOIN BB_BB60.COURSE_MAIN cm on cc.CRSMAIN_PK1 = cm.PK1
    JOIN BB_BB60.GRADEBOOK_MAIN gm on cc.PK1 = gm.COURSE_CONTENTS_PK1
    LEFT JOIN BB_BB60.QTI_ASI_DATA qti on gm.QTI_ASI_DATA_PK1 = qti.PK1
    LEFT JOIN BB_BB60.COURSE_ASSESSMENT ca on qti.PK1 = ca.QTI_ASI_DATA_PK1
    LEFT JOIN BB_BB60.AVL_RULE ar on cc.PK1 = ar.COURSE_CONTENTS_PK1
    LEFT JOIN BB_BB60.AVL_CRIT ac on ar.PK1 = ac.AVL_RULE_PK1
    left join (select crsmain_pk1, count(pk1) students
    from bb_bb60.COURSE_USERS
    where role='S'
    and row_status = 0
    group by crsmain_pk1) students on students.crsmain_pk1 = cm.pk1
    left JOIN (select cu.CRSMAIN_PK1,
                      listagg(u.email, ';') WITHIN GROUP (order by u.email) emails,
                      listagg(u.user_id, ';') WITHIN GROUP (order by u.email) userNames
        from bb_bb60.COURSE_USERS cu join bb_bb60.USERS u on cu.USERS_PK1 = u.PK1
        WHERE (role='P' OR role='T') and
              cu.ROW_STATUS = 0
              group by cu.CRSMAIN_PK1) instructors on instructors.crsmain_pk1 = cm.pk1
    LEFT JOIN (SELECT qti1.PK1 pk1, COUNT(qti3.pk1) questions
        from bb_bb60.QTI_ASI_DATA qti1
        JOIN bb_bb60.QTI_ASI_DATA qti2 on qti1.PK1 = qti2.PARENT_PK1
        JOIN bb_bb60.QTI_ASI_DATA qti3 on qti2.PK1 = qti3.PARENT_PK1
    WHERE qti1.PARENT_PK1 IS NULL AND QTI3.BBMD_ASI_TYPE = 3
    GROUP BY qti1.PK1) questions on gm.QTI_ASI_DATA_PK1 = questions.pk1
     WHERE cm.COURSE_ID LIKE '4206%'
      and cc.CNTHNDLR_HANDLE IN ('resource/x-bb-asmt-survey-link',
                                 'resource/x-bb-assignment',
                                 'resource/x-bb-asmt-test-link')
  and (ar.Title is NULL or ar.Title != 'Test Option Exception Rule')
  and (gm.DUE_DATE > sysdate or gm.DUE_DATE IS NULL))
    WHERE rnk = 1
ORDER BY DUE_DATE
"""

In [18]:
pd.read_sql(q, con=engine).head()

,course_id,course_name,emails,usernames,start_date,end_date,hours_open,due_date,title,available_ind,...,students,cnthndlr_handle,rule_type,pk1,dtcreated,dtmodified,multiple_attempts,force_completion_ind,time_limit,questions
0,4206-74142,2020Summer-MATH 220 Applied Differential Equat...,latricia.miller@ku.edu;myoh@ku.edu,l956m095;myoh12,2020-07-13 12:00:00,2020-07-13 18:00:00,6.0,2020-07-13 16:00:00,Quiz 13_Due_7_13,Y,...,37.0,resource/x-bb-asmt-test-link,S,8295222,2020-05-17 09:50:21,2020-07-11 13:33:38,2,N,40.0,2.0
1,4206-74142,2020Summer-MATH 220 Applied Differential Equat...,latricia.miller@ku.edu;myoh@ku.edu,l956m095;myoh12,NaT,NaT,NaN,2020-07-13 16:00:00,Quiz 13_Due_7_13,Y,...,37.0,resource/x-bb-asmt-test-link,S,8295222,2020-05-17 09:50:21,2020-07-11 13:33:38,2,N,40.0,2.0
2,4206-75320,2020Summer-EAP 101 English for Academic Purpos...,cbuchheit@ku.edu;dtaveg@ku.edu,c083m710;dtaveg,2020-07-13 08:00:00,2020-07-13 16:30:00,8.5,2020-07-13 16:30:00,EAP101_MakingConnections3_Ch4_Reading Test Que...,Y,...,5.0,resource/x-bb-asmt-test-link,None,8411720,2020-07-10 11:10:38,2020-07-10 11:11:50,1,N,50.0,9.0
3,4206-75320,2020Summer-EAP 101 English for Academic Purpos...,cbuchheit@ku.edu;dtaveg@ku.edu,c083m710;dtaveg,2020-07-13 08:00:00,2020-07-13 16:30:00,8.5,2020-07-13 16:30:00,EAP101_MakingConnections3_Ch4_Reading Test Pas...,Y,...,5.0,resource/x-bb-asmt-test-link,None,8411713,2020-07-10 11:08:03,2020-07-10 11:10:17,1,N,50.0,1.0
4,4206-75313,2020Summer-EVRN 170 Introduction to Kansas Lan...,alibrox@ku.edu;harjmand@ku.edu;rhagen@ku.edu,alibrox;h184a117;rhagen,NaT,NaT,NaN,2020-07-13 17:00:00,Home Essay Revision,Y,...,6.0,resource/x-bb-assignment,None,8402290,2020-07-06 11:55:16,2020-07-06 11:55:45,3,None,NaN,NaN
